In [5]:
import os, sys
import glob
import numpy as np
sys.path.append(os.path.join(os.getenv('LAMBO_HOME'), 'lambo/scripts/'))

from hsc_gaap.gaap import findReducedPatches
from hsc_gaap.check_gaap_run import checkRun


In [8]:
tracts = [8282, 8283, 8284, 8285, 8286, 9326, 9327, 9328, 9329, 8522, 8523, 8526, 8527, 8768, 8769, 9010, 9011, 9071, 9072, 9073,
          9074, 9075, 9076, 9077, 9078, 9079, 9080, 9081, 9082, 9083, 9084, 9086, 9086, 9087, 9088, 9089, 9090, 9091, 9092, 9096,
          9097, 9098, 9099, 9100, 9101, 9102, 9103, 9104, 9131, 9132, 9133, 9134, 9135, 9136, 9226, 9227, 9313, 9314, 9315, 9316, 
          9317, 9318, 9319, 9320, 9321, 9322, 9323, 9324, 9325, 9330, 9331, 9332, 9333, 9334, 9335, 9336, 9339, 9340, 9341, 9342,  
          9343, 9344, 9345, 9346, 9347, 9373, 9374, 9375, 9376, 9377, 9378, 9379, 9455, 9456, 9457, 9458, 9459, 9465, 9466, 9467,
          9468, 9469, 9470, 9471, 9556, 9557, 9558, 9559, 9560, 9561, 9562, 9563, 9564, 9565, 9568, 9586, 9587, 9594, 9813, 9836, 
          10059, 10284]

In [18]:
tracts[84]

9347

In [19]:
for tract in tracts[57:85]:
    problems = checkRun(tract)

TRACT: 9314
NO PROBLEMS

TRACT: 9315
NO PROBLEMS

TRACT: 9316
NO PROBLEMS

TRACT: 9317
NO PROBLEMS

TRACT: 9318
NO PROBLEMS

TRACT: 9319
NO PROBLEMS

TRACT: 9320
NO PROBLEMS

TRACT: 9321
NO PROBLEMS

TRACT: 9322
NO PROBLEMS

TRACT: 9323
NO PROBLEMS

TRACT: 9324
NO PROBLEMS

TRACT: 9325
NO PROBLEMS

TRACT: 9330
NO PROBLEMS

TRACT: 9331
NO PROBLEMS

TRACT: 9332
NO PROBLEMS

TRACT: 9333
NO PROBLEMS

TRACT: 9334
NO PROBLEMS

TRACT: 9335
NO PROBLEMS

TRACT: 9336
NO PROBLEMS

TRACT: 9339
NO PROBLEMS

TRACT: 9340
NO PROBLEMS

TRACT: 9341
NO PROBLEMS

TRACT: 9342
NO PROBLEMS

TRACT: 9343
NO PROBLEMS

TRACT: 9344
NO PROBLEMS

TRACT: 9345
NO PROBLEMS

TRACT: 9346
NO PROBLEMS

TRACT: 9347
NO PROBLEMS



In [7]:
problems = checkRun(9081)

TRACT: 9081
NO PROBLEMS



In [54]:
48%9,48//9

(3, 5)

In [34]:
len(problems)

0

In [35]:
print(", ".join(np.array(problems).astype(str)))

---

# Fixing blending matching

In [101]:
%load_ext autoreload
%autoreload 2

In [102]:
from hsc_gaap.run_gaap import matchBlendedness

yay


In [11]:
blenderror ='Traceback (most recent call last):\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 198, in <module>\n    fire.Fire(runGaapRowColumn)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 141, in Fire\n    component_trace = _Fire(component, args, parsed_flag_args, context, name)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 475, in _Fire\n    component, remaining_args = _CallAndUpdateTrace(\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 691, in _CallAndUpdateTrace\n    component = fn(*varargs, **kwargs)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 133, in runGaapRowColumn\n    matchBlendedness(tract, patch_cols, patch_rows, repo=repo)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 187, in matchBlendedness\n    abs(gaapCat["g_ap03Flux"] - blendCat["g_apertureflux10_S20A"][ind]) < 10)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/astropy/table/table.py", line 1872, in __getitem__\n    return self.columns[item]\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/astropy/table/table.py", line 246, in __getitem__\n    return OrderedDict.__getitem__(self, item)\nKeyError: \'g_ap03Flux\''
repo = "/scratch/gpfs/am2907/Merian/gaap/"

In [8]:
def fixBlendMatching(tract, repo = '/scratch/gpfs/am2907/Merian/gaap/', blenderror=blenderror):
    
    # Find patches that didn't match
    print(f'TRACT: {tract}')
    problem_patches=[]

    patches = findReducedPatches(tract)
    dir = os.path.join(repo, f"log/{tract}/")
    logs = np.array(glob.glob(dir + "*.o"))
    logs_patches = np.array([log.split("/")[-1].split(".")[0] for log in logs]).astype(int)
    logs, logs_patches = logs[logs_patches.argsort()], logs_patches[logs_patches.argsort()]
    
    if len(logs) != len(patches):
        print("MISSING PATCHES")

    error3 = "KeyError: 'g_ap03Flux'"
    for i, log in enumerate(logs):
        logfile = open(log, "r").read()

        if error3 in logfile:
            problem_patches.append(logs_patches[i])

    for patch in problem_patches:
        print(f'PATCH: {patch}')
        log = logs[logs_patches==patch][0]
        logfile = open(log, "r").read()
        message = matchBlendedness(tract, [patch%9], [patch//9], repo=repo, matchdist=0.5, matchmag=10)
        newlog_cntnt = logfile.split(blenderror)[0] + f"\n {message} \n" + logfile.split(blenderror)[1]
        newlog = open(log, "w")
        newlog.write(newlog_cntnt)
        newlog.close()

In [129]:
tract=8526

In [13]:
checkRun(9010)

TRACT: 9010
PROBLEM IN PATCH 0: Catalog not saved
PROBLEM IN PATCH 0: Failed for 5 bands
PROBLEM IN PATCH 1: Failed for 3 bands
PROBLEM IN PATCH 2: Failed for 2 bands



array([0, 1, 2])

In [10]:
# fixBlendMatching(8769)

TRACT: 8769


---

In [12]:
tract = 9081
repo = '/scratch/gpfs/am2907/Merian/gaap/'

In [41]:

patches = findReducedPatches(tract)
dir = os.path.join(repo, f"log/{tract}/")
logs = np.array(glob.glob(dir + "*.o"))
logs_patches = np.array([log.split("/")[-1].split(".")[0] for log in logs]).astype(int)
logs, logs_patches = logs[logs_patches.argsort()], logs_patches[logs_patches.argsort()]

In [46]:
logs_patches

array([54, 55, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 72,
       73, 74, 75, 76, 77, 78, 79, 80])

In [44]:
patches

array([54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       72, 73, 74, 75, 76, 77, 78, 79, 80])

---

In [4]:
from astropy.table import Table

In [21]:
tract = 8280
patch = 72

In [20]:
63 %9, 63//9

(0, 7)

In [22]:
test = Table.read(repo + f"S20A/gaapTable/{tract}/{patch%9},{patch//9}/objectTable_{tract}_{patch%9},{patch//9}_S20A.fits")

patch = 63
test2 = Table.read(repo + f"S20A/gaapTable/{tract}/{patch%9},{patch//9}/objectTable_{tract}_{patch%9},{patch//9}_S20A.fits")

In [26]:
len(test.colnames)

949

In [28]:
len(test2.colnames)

949

In [34]:
c2 = test2.colnames
c2.sort()

In [35]:
c1 = test.colnames
c1.sort()

In [38]:
c2==c1

True